<a href="https://colab.research.google.com/github/Joshua-Woodard/Text_Generation_RNN_GRU/blob/main/Text_Generation_RNN_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Text Generation using RNN (LSTM)

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time

In [ ]:
# Download dataset (Shakespeare)
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1130496/1115394 [==============================] - 0s 0us/step


In [ ]:
# Read, then decode for py2 compatibility
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [ ]:
# Examine data
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
# How many unique characters in file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


### Vectorize text

In [ ]:
# Vectorize Text
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), 
    mask_token=None
)

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

def text_from_ids(ids):
  return tf.string.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [ ]:
for ids in ids_dataset.take(10):
  print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [ ]:
seq_length = 100

In [ ]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [ ]:
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)

### Create training batches

In [ ]:
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### Build the model

In [ ]:
# Length of the vocabulary in StringLookup layer
vocab_size = len(ids_from_chars.get_vocabulary())

# Embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [ ]:
model.compile(loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer="adam",
              metrics=["accuracy"])

In [ ]:
# configure checkpoints
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [ ]:
history_txtgen = model.fit(dataset,
                           epochs=30,
                           callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 13s 43ms/step - loss: 2.7057 - accuracy: 0.2794
Epoch 2/30
172/172 [==============================] - 8s 41ms/step - loss: 1.9812 - accuracy: 0.4196
Epoch 3/30
172/172 [==============================] - 8s 40ms/step - loss: 1.6983 - accuracy: 0.4958
Epoch 4/30
172/172 [==============================] - 8s 40ms/step - loss: 1.5390 - accuracy: 0.5375
Epoch 5/30
172/172 [==============================] - 8s 41ms/step - loss: 1.4428 - accuracy: 0.5625
Epoch 6/30
172/172 [==============================] - 8s 41ms/step - loss: 1.3758 - accuracy: 0.5792
Epoch 7/30
172/172 [==============================] - 8s 41ms/step - loss: 1.3245 - accuracy: 0.5920
Epoch 8/30
172/172 [==============================] - 8s 41ms/step - loss: 1.2798 - accuracy: 0.6032
Epoch 9/30
172/172 [==============================] - 8s 41ms/step - loss: 1.2391 - accuracy: 0.6138
Epoch 10/30
172/172 [==============================] - 8s 41ms/step - loss: 1.1999 - accur

### Generate Text

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model,chars_from_ids, ids_from_chars)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['JOSHUA:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end=time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun Time:', end - start)

JOSHUA:
I swear, my lord, where shame doth call me.

CLAUDIO:
Thus might you have, better do another duke.
Dost thou behold the oath debt with the
teeth and the duke is was but newd, and baskly, and
then the red of the world's hand, and with his good as withstard
By and by colour'd in his ere I'll acquit myself:
A creature is possessed with all.

SICINIUS:
To except my horse,
I mean, his a mortal fortune of my fortune!
Hath she use med son.

DUCHESS OF YORK:
O, shadows than ever thou, unsweat hath!
Where is my Roman what is thy house, 'tis round;
Happy wife and mine and low shall
The Lord Hastings, then beloved is hand.

BAPTISTA:
I might take her to prison! lay both they
are choice: then we shall ever be gone.
Within this noble heart love notice,
He must be, this hand and many lips,
Of breach ower to again the rock.

CORIOLANUS:
I mude away,
If not my deceive any fair discreatures be
full-poping true without all, and much lengt
My succession with most pawn'd him hides
His regignce is 

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['JOSHUA:', 'ADAM:', 'BRANDON:', 'RICHARD:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"JOSHUA:more! what's thy name?\n\nBENVOLIO:\nRomeo! my cousin!--\nAll his friends whither away. But he hath something me\nAnd say 'What is't your books: which, vilau-dea,\nand that the sun that love in sight,\nAnd now comfost and in company,\nNo imposting eyes for thee, nor any man?\nShall I not hear it? I mean, he hasks it:\nIf the instance we demand, and to be obstered\nin the time of Gloucester and the key,\nThat longing hours hide more voices of men:\nhis rascul arms was in rut of heart.\nKing Clifford and Bolingbroke!\n\nKING RICHARD III:\nCall it not present to inquire means\nTo pluck the window-blow thou art.\n\nGREMIO:\nWhy should you bide? O that I were as for a world;\nI'll bring what is the whole language and tear-long dut\nLord Angelo is vow a subtle out.\n\nDUCHESS OF YORK:\nO, let me live in thy raise, what have done\nIt shall be parch'd thee to the crown.\n\nKING RICHARD II:\nHere would inclinent to my body to turn\nThe base issue of the death of kings.\nMy 

### Save Generator


In [ ]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

INFO:tensorflow:Assets written to: one_step/assets


INFO:tensorflow:Assets written to: one_step/assets


#### NOTE!

If need more in-depth/customized training for text generator, see https://www.tensorflow.org/text/tutorials/text_generation